In [ ]:
from skimage.io import imread_collection
from skimage.transform import resize
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

def load_images(class_name, values, labels):
    path_to_image = 'raw-img/'
    postfix = '/*.jpeg'
    image_size = (128,128)
    for image in imread_collection(path_to_image + class_name + postfix):
        values.append(resize(image, image_size))
        labels.append(class_name)

def create_data_set():
    class_names = ['spider', 'horse', 'elephant', 'chicken', 'sheep']
    values = []
    labels = []
    for class_name in class_names:
        load_images(class_name, values, labels)

    return np.array(values), np.array(labels)

x, y = create_data_set()
 
class HogTransform(BaseEstimator, TransformerMixin):
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        return np.array([ 
            hog(img,
                orientations=self.orientations,
                pixels_per_cell=self.pixels_per_cell,
                cells_per_block=self.cells_per_block, channel_axis=-1) for img in X ])


In [3]:
hog_pipeline = Pipeline([
    ('hog_transform', HogTransform()),
    ('scaler', StandardScaler()),
    ('logistic',KNeighborsClassifier(n_neighbors=6))
])
 
param_grid = [
    {
        'hog_transform__orientations': [7, 8, 9, 10, 11],
        'hog_transform__cells_per_block': [(2, 2), (3, 3), (4,4), (5,5), (6,6), (7,7), (8,8)],
        'hog_transform__pixels_per_cell': [(6,6), (7,7), (8, 8), (9,9), (10, 10), (12, 12)]
    },

]
search = GridSearchCV(hog_pipeline, param_grid, n_jobs=2, verbose=10)
search.fit(x, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Fitting 5 folds for each of 210 candidates, totalling 1050 fits
[CV 1/5; 1/210] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 2/5; 1/210] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 2/5; 1/210] END hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6);, score=0.466 total time=  34.0s
[CV 3/5; 1/210] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 1/5; 1/210] END hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6);, score=0.462 total time=  35.2s
[CV 4/5; 1/210] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 4/5; 1/210] END hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_